In [1]:
import pandas as pd

In [2]:
prefix = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/'
url = f'{prefix}/yellow_tripdata_2021-01.csv.gz'
df = pd.read_csv(url, nrows=100)

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5


In [5]:
df = pd.read_parquet('./green_tripdata_2025-11.parquet')

In [6]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [8]:
df.columns.tolist()

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge',
 'cbd_congestion_fee']

In [9]:
november_short_trips = df[
    (df['lpep_pickup_datetime'] >= '2025-11-01') & 
    (df['lpep_pickup_datetime'] < '2025-12-01') & 
    (df['trip_distance'] <= 1)
]

In [10]:
count = len(november_short_trips)

In [11]:
print(count)

8007


In [12]:
# Filter for trips with trip_distance < 100 miles
df_filtered = df[df['trip_distance'] < 100].copy()

# Extract the date from lpep_pickup_datetime
df_filtered['pickup_date'] = pd.to_datetime(df_filtered['lpep_pickup_datetime']).dt.date

# Group by pickup_date and find the maximum trip_distance for each day
longest_trip_per_day = df_filtered.groupby('pickup_date')['trip_distance'].max()

# Find the date with the longest trip
day_with_longest_trip = longest_trip_per_day.idxmax()

print(day_with_longest_trip)

2025-11-14


In [13]:
df2 = pd.read_csv('./taxi_zone_lookup.csv')

In [14]:
df2.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [15]:
import pandas as pd

# First, filter the trip data for November 18th, 2025
df_nov18 = df[pd.to_datetime(df['lpep_pickup_datetime']).dt.date == pd.to_datetime('2025-11-18').date()]

# Merge with the location lookup table on PULocationID
df_merged = df_nov18.merge(df2, left_on='PULocationID', right_on='LocationID', how='left')

# Group by Zone and sum the total_amount
zone_totals = df_merged.groupby('Zone')['total_amount'].sum().reset_index()

# Sort by total_amount descending to find the largest
zone_totals_sorted = zone_totals.sort_values('total_amount', ascending=False)

print(zone_totals_sorted.head())

# Get the zone with the largest total_amount
answer = zone_totals_sorted.iloc[0]['Zone']
print(f"\nPickup zone with largest total_amount: {answer}")

                         Zone  total_amount
39          East Harlem North       9281.92
40          East Harlem South       6696.13
23               Central Park       2378.79
128  Washington Heights South       2139.05
85        Morningside Heights       2100.59

Pickup zone with largest total_amount: East Harlem North


In [16]:
east_harlem_north_id = df2[df2['Zone'] == 'East Harlem North']['LocationID'].values[0]

# Filter, merge, and find max tip by drop-off zone
result = (df[
    (pd.to_datetime(df['lpep_pickup_datetime']).dt.to_period('M') == '2025-11') &
    (df['PULocationID'] == east_harlem_north_id)
]
.merge(df2, left_on='DOLocationID', right_on='LocationID')
.groupby('Zone')['tip_amount']
.max()
.sort_values(ascending=False))

print(result.head(10))
print(f"\nAnswer: {result.idxmax()}")

Zone
Yorkville West                   81.89
LaGuardia Airport                50.00
East Harlem North                45.00
Long Island City/Queens Plaza    34.25
JFK Airport                      23.53
Clinton East                     20.00
East Harlem South                20.00
Morningside Heights              20.00
Newark Airport                   20.00
Queens Village                   18.00
Name: tip_amount, dtype: float64

Answer: Yorkville West
